# Bond Valuation

### Lecture Notes by Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MA 195L.2: Introduction to Financial Mathematics II \
Second Semester, S.Y. 2020-2021 \
Ateneo de Manila University

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
def get_tenor(
    start_date, end_date
    ):
    """
    Calculates the tenor in years from start_date to end_date following Actual/360 day count convention
    assumes all input dates are datetime.datetime objects
    """
    
    day_count = (end_date - start_date).days
    tenor = day_count / 360.0
    
    return tenor

In [11]:
def get_bond_price_using_yield(
    today, mat, FV, coupon_rate, coupon_freq, y 
    ):
    """
    Calculates the (clean) price of a COUPON-BEARING (plain-vanilla) bond [not zero-coupon bond] today
    using the given CONTINUOUSLY COMPOUNDED market yield
    
    Inputs:
    today = date today in '%Y-%m-%d' format
    mat =n maturity date in '%Y-%m-%d' format
    Face Value or par value, FV in units of currency
    (annual) coupon_rate in decimal
    coupon_freq = number of times coupon interest is paid in one year
    market yield, y in decimal, continuously compounded
    """
    coupon = (coupon_rate * FV) / coupon_freq
    
    today = dt.datetime.strptime(today, '%Y-%m-%d')
    mat_date = dt.datetime.strptime(mat, '%Y-%m-%d') 
    
    # create list of coupon dates
    # currently only works for annual coupon payments
    coupon_dates = []
    coupon_date = mat_date
    
    while today < coupon_date:
        if coupon_freq==1:# annual
            coupon_dates.append(coupon_date)
            coupon_date = coupon_date.replace(year=coupon_date.year - 1) 
    
    # arrange coupon_dates in increasing order of dates
    coupon_dates = coupon_dates[::-1]
    
    pmt_df = pd.DataFrame({'coupon_dates':coupon_dates})
    
    pmt_df['t'] = [get_tenor(today, t) for t in pmt_df['coupon_dates']]
     
    return pmt_df

In [12]:
get_bond_price_using_yield(
    today="2021-05-04", 
    mat="2025-12-31", 
    FV=1000, 
    coupon_rate=0.10, 
    coupon_freq=1, 
    y=0.11 
    )

,coupon_dates,t
0,2021-12-31,0.669444
1,2022-12-31,1.683333
2,2023-12-31,2.697222
3,2024-12-31,3.713889
4,2025-12-31,4.727778


## Example

Suppose today is May 4, 2021. A 10-year corporate bond has $1000 par value, 10\% coupon bond that pays interest annually, with maturity date on December 31, 2025.  Suppose that the prevailing market yield for the bond is 11\% compounded continuously. Assume an actual/360 day-count convention.

What is the current price of the bond?

Without any other information, we assume that the coupons are paid every December 31 (same day as maturity date). \
Also, since it's a 10-year bond whose maturity date is on December 31, 2025, then the bond was issued last December 31, 2015. 

In [4]:
get_bond_price_using_yield(
    today='2021-05-04', mat='2025-12-31', FV=1000, coupon_rate=0.1, coupon_freq=1, y=0.11
    )

970.72

In [5]:
pmt_df

,coupon_dates,t,cf,df,pv
0,2021-12-31,0.669444,100.0,0.929007,92.900711
1,2022-12-31,1.683333,100.0,0.830966,83.096578
2,2023-12-31,2.697222,100.0,0.743271,74.327109
3,2024-12-31,3.713889,100.0,0.664628,66.462801
4,2025-12-31,4.727778,1100.0,0.594488,653.936272


# Linear Intepolation

Because numpy's interp() function only works when $t\in[x_1, x_2]$ (as seen in the examples below), we created our own linear interpolation function `linear_interp()`. 

An alternative to this would be scipy's [`interpolate.interp1d()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html) function where the parameter `fill_value` is set to "extrapolate".

While we could have performed spline interpolation using all the available zero rates (and not just the two closest available points), we usually just implement linear interpolation in practice for parsimony.

In [17]:
x = [1,2]
y = [0.1025, 0.1033]

t = 0.6694

np.interp(t,x,y)

0.1025

In [19]:
x = [1,2]
y = [0.1025, 0.1033]

t = 1.6833

np.interp(t,x,y)

0.10304664

In [22]:
def linear_interp(t,x,y):
    '''
    Linearly interpolates the zero rate for tenor t 
    using the equation of line determined by the two available points 
    (x_1,y_1) and (x_2,y_2) whose tenors x_1 and x_2 are nearest to t
    
    -------
    Inputs:
    t = tenor of the unknown zero rate 
    x = list [x_1,x_2]
    y = list [y_1,y_2]
    '''
    m = (y[1]-y[0])/(x[1]-x[0])
    
    y = y[0] + m*(t-x[0])
    
    return y

In [23]:
x = [1,2]
y = [0.1025, 0.1033]

t = 1.6833

linear_interp(t,x,y)

0.10304664

In [24]:
x = [1,2]
y = [0.1025, 0.1033]

t = 0.6694

linear_interp(t,x,y)

0.10223552